In [2]:
import psycopg2
import pandas as pd

In [3]:
DBNAME = "opportunity_youth"

In [4]:
conn = psycopg2.connect(dbname=DBNAME)

In [55]:
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tables = []
for table in cursor.fetchall():
    tables.append(table[0])
tables

['pums_2017',
 'puma_names_2010',
 'wa_jobs_2017',
 'wa_geo_xwalk',
 'ct_puma_xwalk']

In [15]:
pums_2017_df = pd.read_sql("SELECT * FROM pums_2017;", conn)
puma_names_2010_df = pd.read_sql("SELECT * FROM puma_names_2010;", conn)
wa_jobs_2017_df = pd.read_sql("SELECT * FROM wa_jobs_2017;", conn)
wa_geo_xwalk_df = pd.read_sql("SELECT * FROM wa_geo_xwalk;", conn)
ct_puma_xwalk_df = pd.read_sql("SELECT * FROM ct_puma_xwalk;", conn)

In [88]:
pd.read_sql("""SELECT * FROM puma_names_2010 WHERE state_name LIKE 'Washington%' AND puma_name LIKE 'King County%';""", conn)

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name
0,53,Washington ...,1044,11606,5311606,G53011606,"King County (Northwest)--Shoreline, Kenmore & ..."
1,53,Washington ...,1044,11607,5311607,G53011607,"King County (Northwest)--Redmond, Kirkland Cit..."
2,53,Washington ...,1044,11608,5311608,G53011608,King County (Northwest Central)--Greater Belle...
3,53,Washington ...,1044,11609,5311609,G53011609,"King County (Central)--Sammamish, Issaquah, Me..."
4,53,Washington ...,1044,11610,5311610,G53011610,"King County (Central)--Renton City, Fairwood, ..."
5,53,Washington ...,1044,11613,5311613,G53011613,King County (Southwest Central)--Kent City ...
6,53,Washington ...,1044,11614,5311614,G53011614,King County (Southwest)--Auburn City & Lakelan...
7,53,Washington ...,1044,11615,5311615,G53011615,"King County (Southeast)--Maple Valley, Covingt..."
8,53,Washington ...,1044,11616,5311616,G53011616,"King County (Northeast)--Snoqualmie City, Cott..."
9,53,Washington ...,1045,11611,5311611,G53011611,"King County (West Central)--Burien, SeaTac, Tu..."


In [66]:
#cow : class of working 9 .Unemployed and last worked 5 years ago or earlier or never
#wkl : When last worked

df_new = pd.read_sql("""
SELECT puma, agep, cow, wkl, sch, schl, rt
FROM pums_2017
WHERE (agep BETWEEN 16.0 AND 24.9)
AND rt = 'P'
AND CAST(sch AS int) = 1 OR sch IS null
AND CAST(cow AS int) = 9
""", conn)
df_new

,puma,agep,cow,wkl,sch,schl,rt
0,11801,17.0,1,1,1,16,P
1,10800,23.0,1,1,1,16,P
2,11802,22.0,5,1,1,16,P
3,10300,19.0,1,1,1,16,P
4,10703,24.0,1,1,1,16,P
...,...,...,...,...,...,...,...
15147,11603,22.0,None,3,1,12,P
15148,11705,23.0,4,2,1,17,P
15149,10702,21.0,None,3,1,17,P
15150,11701,21.0,1,1,1,14,P


In [81]:
df_new = pd.read_sql("""
SELECT serialno, puma, agep, cow, wkl, sch, schl, rt
FROM pums_2017
JOIN puma_names_2010
USING (puma)
WHERE (agep BETWEEN 16.0 AND 24.9)
AND rt = 'P'
AND CAST(sch AS int) = 1 OR sch IS null
AND CAST(cow AS int) = 9
AND state_name LIKE 'Washington%'
AND puma_name LIKE 'King County%'
""", conn)
df_new = df_new.drop_duplicates().reset_index(drop=True)
df_new

,serialno,puma,agep,cow,wkl,sch,schl,rt
0,2013000000181,11801,17.0,1,1,1,16,P
1,2013000000664,10800,23.0,1,1,1,16,P
2,2013000001126,11802,22.0,5,1,1,16,P
3,2013000001584,10300,19.0,1,1,1,16,P
4,2013000001874,10703,24.0,1,1,1,16,P
...,...,...,...,...,...,...,...,...
14910,2017001533434,11705,23.0,4,2,1,17,P
14911,2017001535873,11801,20.0,5,1,1,16,P
14912,2017001534402,11603,22.0,None,3,1,12,P
14913,2017001536310,11701,21.0,1,1,1,14,P
